In [26]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
 
EPOCHS=50
NUM_CLASSES = 10
    

def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train/255
    x_test = x_test/255
 
    #normalize 
    #mean = np.mean(x_train,axis=(0,1,2,3))
    #std = np.std(x_train,axis=(0,1,2,3))
    #x_train = (x_train-mean)/(std+1e-7)
    #x_test = (x_test-mean)/(std+1e-7)
 
    y_train =  tf.keras.utils.to_categorical(y_train,NUM_CLASSES)
    y_test =  tf.keras.utils.to_categorical(y_test,NUM_CLASSES)

    return x_train, y_train, x_test, y_test

def build_model(): 
    model = models.Sequential()
    
    #1st blocl
    model.add(layers.Conv2D(32, (3,3), padding='same', 
        input_shape=x_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))

    #2nd block
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))

    #3d block 
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))

    #dense  
    model.add(layers.Flatten())
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
    return model

    model.summary()
 

(x_train, y_train, x_test, y_test) = load_data()
model = build_model()
model.compile(loss='categorical_crossentropy', 
            optimizer='RMSprop', 
            metrics=['accuracy'])

#image augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )
datagen.fit(x_train)
 
#train
batch_size = 64
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=EPOCHS,
                    verbose=1,validation_data=(x_test,y_test))
#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5') 

#test
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0])) 

Epoch 1/50


<ipython-input-26-2236a13b2652>:79: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),


782/782 [==============================] - 30s 36ms/step - loss: 2.0740 - accuracy: 0.3753 - val_loss: 8.6824 - val_accuracy: 0.2872
Epoch 2/50
782/782 [==============================] - 28s 35ms/step - loss: 1.6293 - accuracy: 0.4965 - val_loss: 1.2350 - val_accuracy: 0.5741
Epoch 3/50
782/782 [==============================] - 28s 35ms/step - loss: 1.4483 - accuracy: 0.5405 - val_loss: 1.2450 - val_accuracy: 0.5881
Epoch 4/50
782/782 [==============================] - 29s 36ms/step - loss: 1.2727 - accuracy: 0.5856 - val_loss: 1.0598 - val_accuracy: 0.6556
Epoch 5/50
782/782 [==============================] - 29s 36ms/step - loss: 1.1641 - accuracy: 0.6163 - val_loss: 1.2333 - val_accuracy: 0.6067
Epoch 6/50
782/782 [==============================] - 28s 36ms/step - loss: 1.0781 - accuracy: 0.6361 - val_loss: 1.0539 - val_accuracy: 0.6727
Epoch 7/50
782/782 [==============================] - 28s 35ms/step - loss: 1.0140 - accuracy: 0.6528 - val_loss: 1.1567 - val_accuracy: 0.6503
Epo